In [1]:
import db데이터처리 as db
import pandas as pd
from 모델 import GCN_Model
import numpy as np

In [37]:
real_price = pd.read_csv('../000660_11.csv')
real_price = real_price['종가']
real_price.to_excel('./11일종가.xlsx')

In [51]:
true_result = pd.read_csv('../000660_11이진분류Vector.csv')
true_result = true_result['18']
# true_result.to_excel('./11일답.xlsx')

In [52]:
true_result = true_result.astype(float)
true_result = np.insert(true_result, 0, [np.nan] * 12)
true_result = pd.DataFrame(true_result)

In [53]:
true_result

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
375,1.0
376,1.0
377,1.0
378,1.0


In [54]:
true_df = pd.concat([real_price, true_result],axis=1)

In [55]:
true_df

,종가,0
0,156800,NaN
1,156800,NaN
2,156700,NaN
3,156600,NaN
4,156900,NaN
...,...,...
375,156300,1.0
376,156400,1.0
377,156400,1.0
378,156500,1.0


In [2]:
data, close = db.get_vector_list('000660', 1461, 0)

1460   2024-09-11 09:10:00
Name: Jdate, dtype: datetime64[ns]


In [64]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,6.2578,-18.7852,-6.2500,6.2461,-25.0000,0.0000,0.0000,6.2344,0.0000,-1.50,-15.17,2.01,6.84,-9.22,3.19,2.54,7.93,-1.76
1,-18.7852,-6.2500,6.2461,-25.0000,0.0000,0.0000,6.2344,0.0000,6.2383,-15.17,2.01,6.84,-9.22,3.19,2.54,7.93,-1.76,0.36
2,-6.2500,6.2461,-25.0000,0.0000,0.0000,6.2344,0.0000,6.2383,18.7266,2.01,6.84,-9.22,3.19,2.54,7.93,-1.76,0.36,5.95
3,6.2461,-25.0000,0.0000,0.0000,6.2344,0.0000,6.2383,18.7266,-6.2539,6.84,-9.22,3.19,2.54,7.93,-1.76,0.36,5.95,-11.07
4,-25.0000,0.0000,0.0000,6.2344,0.0000,6.2383,18.7266,-6.2539,-6.2500,-9.22,3.19,2.54,7.93,-1.76,0.36,5.95,-11.07,-2.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,0.0000,6.4516,-122.6598,0.0000,6.3776,6.3816,-19.1571,19.1205,-6.3857,-14.98,9.72,-155.08,142.53,-4.60,0.57,-8.91,23.76,-16.98
287,6.4516,-122.6598,0.0000,6.3776,6.3816,-19.1571,19.1205,-6.3857,-25.5265,9.72,-155.08,142.53,-4.60,0.57,-8.91,23.76,-16.98,-6.71
288,-122.6598,0.0000,6.3776,6.3816,-19.1571,19.1205,-6.3857,-25.5265,6.3654,-155.08,142.53,-4.60,0.57,-8.91,23.76,-16.98,-6.71,8.63
289,0.0000,6.3776,6.3816,-19.1571,19.1205,-6.3857,-25.5265,6.3654,6.3694,142.53,-4.60,0.57,-8.91,23.76,-16.98,-6.71,8.63,-11.86


In [65]:
a = int(close['종가'][-1:].values)

In [66]:
a

156800

In [27]:
pred = GCN_Model(data)

예측 : 1


In [3]:

pred_list=[]
# 백테스팅
# 초기 자본금
initial_capital = 200000
capital = initial_capital
position = 0  # 현재 보유 주식 수
buy_price = 0  # 매수 가격
sell_count = 0  # 총 매도 횟수
buy_count = 0  # 총 매수 횟수
sell_fee = 0.00195
buy_fee = 0.00015
zero_count = 0
fee = 0 # 수수료

# 결과를 저장할 리스트
results = []
tmp = None
for i in range(368):
    data, close = db.get_vector_list('000660', 1461+i,0 )
    pred = GCN_Model(data)
    pred_list.append(pred)
    tmp = close
    # 수익률 계산

    # if pred == 1 and position == 0:  # 매수 조건
    #     position = capital // int(close['종가'][-1:].values)  # 최대 매수 가능 수량
    #     buy_price = int(close['종가'][-1:].values)  # 매수 가격
    #     buy_count += 1  # 매수 횟수
    #     fee += (buy_price * position) * buy_fee  # 매수 수수료
    #     capital -= (position * buy_price) + ((buy_price * position) * buy_fee)  # 자본금 감소
    #     results.append({'행동': '매수', '가격': buy_price, '잔고': capital})

    # elif pred == 0 and position != 0 and zero_count == 2:  # 매도 조건
    #     sell_price = int(close['종가'][-1:].values)  # 매도 가격
    #     capital += (position * sell_price) - ((sell_price * position) * sell_fee)  # 매도 후 자본금 증가
    #     fee += (sell_price * position) * sell_fee  # 매도 수수료
    #     sell_count += 1  # 매도 횟수
    #     zero_count = 0
        
    #     # 수익률 계산
    #     profit_per_share = sell_price - buy_price
    #     profit_rate = profit_per_share-buy_price
        
    #     results.append({'행동': '매도', '가격': sell_price, '잔고': capital, '수익금': profit_per_share})
    #     position = 0  # 보유 주식 수 초기화
        
    # elif pred == 0 and position != 0:
    #     zero_count += 1
    #     print(zero_count)
    
    
    # 3진분류
    # if pred == 2 and position == 0:  # 매수 조건
    #     position = capital // int(close['종가'][-1:].values)  # 최대 매수 가능 수량
    #     buy_price = int(close['종가'][-1:].values)  # 매수 가격
    #     buy_count += 1  # 매수 횟수
    #     fee += (buy_price * position) * buy_fee  # 매수 수수료
    #     capital -= (position * buy_price) + ((buy_price * position) * buy_fee)  # 자본금 감소
    #     results.append({'행동': '매수', '가격': buy_price, '잔고': capital})

    # elif pred == 0 and position > 0 and zero_count == 2:  # 매도 조건
    #     sell_price = int(close['종가'][-1:].values)  # 매도 가격
    #     capital += (position * sell_price) - ((sell_price * position) * sell_fee)  # 매도 후 자본금 증가
    #     fee += (sell_price * position) * sell_fee  # 매도 수수료
    #     sell_count += 1  # 매도 횟수
    #     zero_count = 0
                
    #     # 수익률 계산
    #     profit_per_share = sell_price - buy_price
    #     profit_rate = profit_per_share-buy_price
        
    #     results.append({'행동': '매도', '가격': sell_price, '잔고': capital, '수익금': profit_per_share})
    #     position = 0  # 보유 주식 수 초기화
    
    # elif pred == 0 and position != 0:
    #     zero_count += 1

# # 결과를 데이터프레임으로 변환
# results_df = pd.DataFrame(results)
# # results_df.to_excel('./하이닉스수익확인.xlsx')
# # 최종 자산 가치 계산
# final_value = capital + position * int(tmp['종가'][-1:].values)
# profit = final_value - initial_capital
# profit_rate = (profit / initial_capital) * 100

# print(f'초기 자본금: {initial_capital}원')
# print(f'최종 자산 가치: {final_value:.2f}원')
# print(f'총 수익: {profit:.2f}원')
# print(f'수익률: {profit_rate:.2f}%')
# print(f'매도 횟수: {sell_count}, 매수 횟수: {buy_count}')
# print(f'총 수수료 : {fee:.2f}원')

# # 결과 출력
# results_df

1460   2024-09-11 09:10:00
Name: Jdate, dtype: datetime64[ns]
예측 : 0
1461   2024-09-11 09:11:00
Name: Jdate, dtype: datetime64[ns]
예측 : 1
1462   2024-09-11 09:12:00
Name: Jdate, dtype: datetime64[ns]
예측 : 2
1463   2024-09-11 09:13:00
Name: Jdate, dtype: datetime64[ns]
예측 : 0
1464   2024-09-11 09:14:00
Name: Jdate, dtype: datetime64[ns]
예측 : 0
1465   2024-09-11 09:15:00
Name: Jdate, dtype: datetime64[ns]
예측 : 0
1466   2024-09-11 09:16:00
Name: Jdate, dtype: datetime64[ns]
예측 : 0
1467   2024-09-11 09:17:00
Name: Jdate, dtype: datetime64[ns]
예측 : 0
1468   2024-09-11 09:18:00
Name: Jdate, dtype: datetime64[ns]
예측 : 2
1469   2024-09-11 09:19:00
Name: Jdate, dtype: datetime64[ns]
예측 : 2
1470   2024-09-11 09:20:00
Name: Jdate, dtype: datetime64[ns]
예측 : 0
1471   2024-09-11 09:21:00
Name: Jdate, dtype: datetime64[ns]
예측 : 2
1472   2024-09-11 09:22:00
Name: Jdate, dtype: datetime64[ns]
예측 : 2
1473   2024-09-11 09:23:00
Name: Jdate, dtype: datetime64[ns]
예측 : 2
1474   2024-09-11 09:24:00
Name: J

In [4]:
len(pred_list)

368

In [5]:
pred_list_df = pd.DataFrame(pred_list)
pred_list_df.to_excel('./11일예측3진.xlsx')

In [ ]:
import win32com.client

# 연결 여부 체크
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect
if (bConnect == 0):
    print("PLUS가 정상적으로 연결되지 않음. ")
    exit()

# 주문 초기화
objTrade =  win32com.client.Dispatch("CpTrade.CpTdUtil")

initCheck = objTrade.TradeInit(0)

if (initCheck != 0):
    print("주문 초기화 실패")
    exit()


# 주식 매수 주문
acc = objTrade.AccountNumber[0] #계좌번호
accFlag = objTrade.GoodsList(acc, 1)  # 주식상품 구분
print(acc, accFlag[0])
objStockOrder = win32com.client.Dispatch("CpTrade.CpTd0311")
objStockOrder.SetInputValue(0, "1")   # 2: 매수
objStockOrder.SetInputValue(1, acc )   #  계좌번호
objStockOrder.SetInputValue(2, accFlag[0])   # 상품구분 - 주식 상품 중 첫번째
objStockOrder.SetInputValue(3, "A000660")   # 종목코드 - 필요한 종목으로 변경 필요 
objStockOrder.SetInputValue(4, 1)   # 매수수량 - 요청 수량으로 변경 필요 
objStockOrder.SetInputValue(5, 62600)   # 주문단가 - 필요한 가격으로 변경 필요 
objStockOrder.SetInputValue(7, "0")   # 주문 조건 구분 코드, 0: 기본 1: IOC 2:FOK
objStockOrder.SetInputValue(8, "03")   # 주문호가 구분코드 - 01: 보통

# 매수 주문 요청
nRet = objStockOrder.BlockRequest()
if (nRet != 0) :
    print("주문요청 오류", nRet)    
    # 0: 정상,  그 외 오류, 4: 주문요청제한 개수 초과 
    exit()
    
# if (nRet == 0):
#     print('정상 주문 완료')
#     exit()

rqStatus = objStockOrder.GetDibStatus()
errMsg = objStockOrder.GetDibMsg1()
if rqStatus != 0:
    print("주문 실패: ", rqStatus, errMsg)
    exit()
